# Open notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/arsalmairaj2k/taxi_trip_analysis/blob/main/Task3.ipynb)

# Task 3: Data Preprocessing & Feature Engineering

# Load Dataset
Just like the previous task, we load our dataset from our GitHub repository.

In [40]:
import pandas as pd

GITHUB_URL = "https://github.com/arsalmairaj2k/taxi_trip_analysis/raw/main/TaxiTripData.xlsx"
df = pd.read_excel(GITHUB_URL)

# 1. Handle missing values(drop, fill with median/mean/mode, or use imputation).

## Identify Missing Values

In [42]:
# Checking for missing values
missing_values = df.isnull().sum()

# Display columns with missing values
print("Missing Values per Column:")
print(missing_values[missing_values > 0])

# Total missing values in the dataset
total_missing = df.isnull().sum().sum()
print(f"\nTotal Missing Values in Dataset: {total_missing}")


Missing Values per Column:
VendorID              2131
passenger_count       2131
RatecodeID            2131
store_and_fwd_flag    2131
payment_type          2131
dtype: int64

Total Missing Values in Dataset: 10655


As done in the previous task and also explained, some machine learning algorithms can't perfrom efficiently without complete data so we do what we did in previous task by using imputation provided by sklearn, but this time we replace missing numerical values with median

In [44]:
from sklearn.impute import SimpleImputer

# Selecting only numeric features (excluding timestamps and categorical variables)
numeric_cols = df.select_dtypes(include=['number']).columns

# Using Median Imputation on numeric columns only
imputer = SimpleImputer(strategy='median')
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# Handling missing values in categorical columns with mode (as done before as well)
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df.loc[:, col] = df[col].fillna(df[col].mode()[0])

print(df.head())

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0       1.0  2020-01-01 00:28:00   2020-01-01 00:33:00              1.0   
1       1.0  2020-01-01 00:35:00   2020-01-01 00:43:00              1.0   
2       1.0  2020-01-01 00:47:00   2020-01-01 00:53:00              1.0   
3       1.0  2020-01-01 00:55:00   2020-01-01 01:00:00              1.0   
4       2.0  2020-01-01 00:01:00   2020-01-01 00:04:00              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0            1.2         1.0                  N         238.0         239.0   
1            1.2         1.0                  N         239.0         238.0   
2            0.6         1.0                  N         238.0         238.0   
3            0.8         1.0                  N         238.0         151.0   
4            0.0         1.0                  N         193.0         193.0   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


# 2. Convert categorical features to numerical (One-Hot Encoding or Label Encoding).

In [48]:
# Ensure 'store_and_fwd_flag' is treated as a string before encoding
df['store_and_fwd_flag'] = df['store_and_fwd_flag'].astype(str)

# Apply One-Hot Encoding
df = pd.get_dummies(df, columns=['store_and_fwd_flag'], drop_first=True)

# Convert the new column to integer type (0 and 1)
df['store_and_fwd_flag_Y'] = df['store_and_fwd_flag_Y'].astype(int)

# Display the first few rows to verify the transformation
print(df.head())


   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0       1.0  2020-01-01 00:28:00   2020-01-01 00:33:00              1.0   
1       1.0  2020-01-01 00:35:00   2020-01-01 00:43:00              1.0   
2       1.0  2020-01-01 00:47:00   2020-01-01 00:53:00              1.0   
3       1.0  2020-01-01 00:55:00   2020-01-01 01:00:00              1.0   
4       2.0  2020-01-01 00:01:00   2020-01-01 00:04:00              1.0   

   trip_distance  RatecodeID  PULocationID  DOLocationID  payment_type  \
0            1.2         1.0         238.0         239.0           1.0   
1            1.2         1.0         239.0         238.0           1.0   
2            0.6         1.0         238.0         238.0           1.0   
3            0.8         1.0         238.0         151.0           1.0   
4            0.0         1.0         193.0         193.0           2.0   

   fare_amount  extra  mta_tax  tip_amount  tolls_amount  \
0          6.0    3.0      0.5        1.47  

# 3. Scale numerical features using  MinMax Scalin
Why MinMax Scaling?

It scales all numerical features to a range of 0 and 1, making the data suitable for ML models.g

In [53]:
from sklearn.preprocessing import MinMaxScaler

# Selecting only numeric columns (excluding the target variable 'fare_amount' if needed)
numeric_cols = df.select_dtypes(include=['number']).columns

# Initialize MinMaxScaler (scales values between 0 and 1)
scaler = MinMaxScaler()

# Apply scaling
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Display first few rows to verify
print(df.head())


   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0       0.0  2020-01-01 00:28:00   2020-01-01 00:33:00         0.111111   
1       0.0  2020-01-01 00:35:00   2020-01-01 00:43:00         0.111111   
2       0.0  2020-01-01 00:47:00   2020-01-01 00:53:00         0.111111   
3       0.0  2020-01-01 00:55:00   2020-01-01 01:00:00         0.111111   
4       1.0  2020-01-01 00:01:00   2020-01-01 00:04:00         0.111111   

   trip_distance  RatecodeID  PULocationID  DOLocationID  payment_type  \
0       0.083085         0.0      0.897727      0.901515      0.000000   
1       0.083085         0.0      0.901515      0.897727      0.000000   
2       0.080952         0.0      0.897727      0.897727      0.000000   
3       0.081663         0.0      0.897727      0.568182      0.000000   
4       0.078820         0.0      0.727273      0.727273      0.333333   

   fare_amount     extra   mta_tax  tip_amount  tolls_amount  \
0     0.502423  0.714286  0.263158    0.